In [ ]:
import os
import sys
from google.colab import drive
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import dlib
from scipy import ndimage
import tarfile
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%mkdir -p /content/data/
%cd /content/data

train_path = '/content/drive/My Drive/training/training.tar'
val_path = '/content/drive/My Drive/training/validation.tar'
test_path = '/content/drive/My Drive/training/test.tar'

# Load training set and validation set
for fpath in [train_path, val_path, test_path]:
  print('Extracting {}...'.format(fpath.split('/')[-1]))
  with tarfile.open(fpath) as tar:
    tar.extractall()

/content/data
Extracting training.tar...
Extracting validation.tar...
Extracting test.tar...


## **Image Preprocessing**

In [ ]:
import os

import cv2
import dlib
import numpy as np
from PIL import Image


class align_faces(object):

    def __call__(self, image):

        # Since the images are loaded it PIL
        image = np.asarray(image)

        # The percentage value of how far in the picture the left eye should be
        LEFT_EYE_CORD = (0.25, 0.2)
        DIMENSIONS = 244

        train_folder = '/content/drive/My Drive/training/'
        predictor_path = os.path.join(train_folder, "shape_predictor_5_face_landmarks.dat")
        detector_path = os.path.join(train_folder, "mmod_human_face_detector.dat")
        shape_predictor = dlib.shape_predictor(predictor_path)
        face_detector = dlib.get_frontal_face_detector()

        # changed here for efficiency
        # face_detector = dlib.cnn_face_detection_model_v1(detector_path)
        faces = face_detector(image)

        if not faces:
            raise ValueError("Image has no detectable faces")

        # assumption is made that there is only one
        for face in faces:
          # face = face_rect.rect
          landmarks = shape_predictor(image, face)
          landmarks = landmarks.parts()
          landmarks = self.convert_to_np(landmarks)

          # To Gauge Scale
          maximum = np.max(landmarks, axis=0)
          minimum = np.min(landmarks, axis=0)

          # eye landmarks
          right = landmarks[:2]
          left = landmarks[2:4]

          left = np.mean(left, axis=0, dtype=np.int)
          right = np.mean(right, axis=0, dtype=np.int)

          centre = np.vstack((left, right))
          centre = np.mean(centre, axis=0, dtype=np.int)

          diff = right - left
          diff = diff.reshape((2, 1))

          angle = np.degrees(np.arctan2(diff[1], diff[0]))

          # find the length of the face, and use that for our scale
          y_scale = maximum[1] - minimum[1]
          y_scale = y_scale + 2.1 * y_scale

          M = cv2.getRotationMatrix2D((centre[0], centre[1]), angle, DIMENSIONS / y_scale)

          # update translation
          t_x = DIMENSIONS // 2
          t_y = DIMENSIONS * LEFT_EYE_CORD[1]
          M[0, 2] += (t_x - centre[0])
          M[1, 2] += (t_y - centre[1])

          image2 = cv2.warpAffine(image, M, (DIMENSIONS, DIMENSIONS),
                                  flags=cv2.INTER_CUBIC)

          # convert back to PIL
          return Image.fromarray(image2)

    @staticmethod
    def convert_to_np(points):
        np_points = np.array([], dtype=np.int)
        while points:
            point = points.pop()
            np_points = np.append(np_points, (point.x, point.y))

        np_points = np_points.reshape((-1, 2))
        np_points = np.flip(np_points, axis=0)
        return np_points

In [ ]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.Grayscale(num_output_channels=3), 
     align_faces(),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.3898, 0.3884, 0.3878), (0.2433, 0.2430, 0.2430))
     ]
)

## **Dataset Creation**

In [ ]:
class AUDataset(Dataset):
  """ CK+ dataset labeled by the presence of action units (AU) """
  
  def __init__(self, mode='train', fold=0, transform=None):
    """
    Args:
      - label_csv: Path to the csv file with action unit labels.
      - mode: one of 'train', 'validate', and 'test'
      - fold: one of 0, 1, and 2; determines which folds to use for training
      - transform: transform applied to an image input
    """
    try:
      path_dict = {
          'train': 'training/train_data_',
          'validate': 'validation/valid_data_',
          'test': 'test/test_data',
          'train_full': 'train_full/train_full_data',
          'validate_full': 'val_full/val_full_data'
      }
      fold = fold if ((mode == 'train') or (mode == 'validate')) else ''
      self.data_path = '{}{}'.format(path_dict[mode], fold)
      self.label_path = '{}{}_labels.csv'.format(path_dict[mode], fold)
    except KeyError:
      raise ValueError('{} is not a valid mode. Choose train, validation, or test.'.format(mode))
    self.au_frame = pd.read_csv(self.label_path)
    self.label_cols = ['AU_1', 'AU_2', 'AU_4', 'AU_5', 'AU_6', 'AU_7', 
                        'AU_9', 'AU_10', 'AU_12', 'AU_14', 'AU_15', 'AU_17',
                        'AU_20', 'AU_23/24', 'AU_25', 'AU_26/27', 'AU_28',
                        'AU_43']
    self.transform = transform
    

  def __len__(self):
    return len(self.au_frame)

  def __getitem__(self, idx):
    # Get image at idx
    image_id = self.au_frame.iloc[idx]['path']
    image_path = self.data_path + '/' + image_id
    image = cv2.imread(image_path)
    try:
      image = Image.fromarray(image)
    except AttributeError:
      raise ValueError('{} not found'.format(image_path))

    # Get AU labels
    aus = self.au_frame.iloc[idx][self.label_cols]
    aus = np.array(aus, dtype=float)

    if self.transform:
      try:
        image = self.transform(image)
      except ValueError: # No faces were detected
        return None

    sample = {'image': image, 'labels': aus}

    return sample

## **Other Useful Functions**

In [ ]:
def collate_fn(batch):
    """
    Used to process the list of samples to form a batch. Ignores images
    where no faces were detected.
    """
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap="gray")
    plt.show()

## **Training Loop**

In [ ]:
import copy
import time

import torchvision
from scipy.io.idl import AttrDict
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np


def validation_step(convnet, val_loader, criterion, bs):
  num_matches = 0.0
  total = 0.0
  losses = []
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  with torch.no_grad():
      for _, item in enumerate(val_loader, 0):

          labels = item.get("labels")
          imgs = item.get("image")
          labels = labels.to(device)
          imgs = imgs.to(device)

          outputs = convnet(imgs)

          # Compute batch loss
          val_loss = criterion(outputs, labels)
          losses.append(val_loss.data.item())

          # Compute batch accuracy, set probabilities > 0.5 to 1
          t = torch.Tensor([0.5])
          t = t.to(device)
          num_matches += ((torch.nn.functional.sigmoid(outputs) > t) == labels).sum()

          total += labels.size(0) * 18

  val_loss = np.mean(losses)
  val_acc = 100 * num_matches / total
  return val_loss, val_acc


def train(convnet, args):
  """Training loop for a single validation fold."""
  train_losses = []
  valid_losses = []
  valid_accs = []
  best_loss = -100 
  start = time.time()

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  best_model_wts = copy.deepcopy(convnet.state_dict())

  train_set = AUDataset(mode='train', fold=args.fold, transform=args.transform)
  train_loader = DataLoader(train_set, args.batch_size, collate_fn=collate_fn,
                            shuffle=True, num_workers=args.num_workers)
  
  val_set = AUDataset(mode='validate', fold=args.fold, transform=args.transform)
  val_loader = DataLoader(val_set, args.batch_size, collate_fn=collate_fn,
                          shuffle=True, num_workers=args.num_workers)

  optimizer = torch.optim.Adam(convnet.parameters(), args.learn_rate)
  criterion = torch.nn.BCEWithLogitsLoss()

  if args.resume: 
    print("Loading checkpoint")
    state = torch.load(args.checkpoint)
    convnet.load_state_dict(state['model_state_dict'])
    optimizer.load_state_dict(state['optimizer_state_dict'])


  for epoch in range(args.epochs):
      convnet.train()
      losses = []

      for _, item in enumerate(train_loader, 0):
          
          labels = item.get("labels")
          imgs = item.get("image")

          labels = labels.to(device)
          imgs = imgs.to(device)

          optimizer.zero_grad()

          outputs = convnet(imgs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          losses.append(loss.data.item())


      avg_loss = np.mean(losses)
      train_losses.append(avg_loss)
      time_elapsed = time.time() - start

      print('Epoch [%d/%d], Loss: %.4f, Time (s): %d' % (
          epoch + 1, args.epochs, avg_loss, time_elapsed))
      
      # Validation 
      convnet.eval()
      val_loss, val_acc = validation_step(convnet, val_loader, criterion,
                                          args.batch_size)
      time_elapsed = time.time() - start
      valid_losses.append(val_loss)
      valid_accs.append(val_acc)

      print('Epoch [%d/%d], Val Loss: %.4f, Val Acc: %.1f%%, Time(s): %.2f' % (
          epoch + 1, args.epochs, val_loss, val_acc, time_elapsed))

      # Save model
      if -val_loss >= best_loss:
        print("Best Loss: Saving Model")
        best_loss = -val_loss
        checkpoint = {
              'model_state_dict': convnet.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'train_losses': train_losses,
              'valid_losses': valid_losses,
              'best_val_loss': best_loss
          }
        torch.save(checkpoint, args.checkpoint)


def match_state_dict(state_dict):
  """
  Renames 'fc.0' to 'fc.1' and 'fc.2' to 'fc.3' in order to load pretrained
  model weights to new model.
  """
  new_sd = copy.deepcopy(state_dict)
  for key in state_dict:
    if 'fc' in key:
      layer, num, weight = key.split('.')
      if int(num) == 0:
        new_sd['{}.1.{}'.format(layer, weight)] = new_sd.pop(key)
      elif int(num) == 2:
        new_sd['{}.3.{}'.format(layer, weight)] = new_sd.pop(key)
  return new_sd

def get_model(path):
  """
  Returns a model that has already been pretrained on the CelebA dataset.
  """
  model = models.resnet18(pretrained=True)
  num = model.fc.in_features
  fc = torch.nn.Sequential(
      torch.nn.LeakyReLU(),
      torch.nn.Linear(512, 256), 
      torch.nn.LeakyReLU(),
      torch.nn.Linear(256, 18))
  
  model.fc = fc
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  last_cp = torch.load(path)
  model_state_dict = match_state_dict(last_cp['model_state_dict'])
  model.load_state_dict(model_state_dict)
  model = model.to(device)
  return model

## **Scoring Metrics**

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, auc, precision_recall_curve

def get_metrics(model, data):
  """
  Compute the f1 score for each action unit (AU).
  """
  labels = np.array([], dtype=int).reshape(0,18)
  preds = np.array([], dtype=int).reshape(0,18)
  model.eval() # Set to evaluation mode
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model.to(device)

  with torch.no_grad():
    for _, item in enumerate(data, 0):
      batch_labels = item.get("labels")
      labels = np.vstack([labels, batch_labels])
      imgs = item.get("image")
      imgs = imgs.to(device)
      outputs = model(imgs).cpu().detach()
      outputs = torch.sigmoid(outputs).numpy()
      preds = np.vstack([preds, outputs])
  
  # Compute f1 scores
  f1_scores = []
  pres_scores = []
  recall_scores = []
  pr_auc_scores = []
  for au in range(18):
    au_labels = labels[:, au]
    au_preds = preds[:, au]
    p, r, _ = precision_recall_curve(au_labels, au_preds)
    auc_pr = auc(r, p)
    if np.isnan(auc_pr):
      auc_pr = 0
    pr_auc_scores.append(auc_pr)


    # Threshold probabilities > 0.4 to 1 for other metrics
    if au == 17:
      au_preds = (au_preds > 0.05) * 1
    else:
      au_preds = (au_preds > 0.4) * 1
    f1 = f1_score(au_labels, au_preds, zero_division=0); f1_scores.append(f1)
    precision = precision_score(au_labels, au_preds, zero_division=0)
    pres_scores.append(precision)
    recall = recall_score(au_labels, au_preds, zero_division=0)
    recall_scores.append(recall)
  return pres_scores, recall_scores, f1_scores, pr_auc_scores


## **First Fold**: Unfreeze only FC layers

In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint0.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=0, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.754, recall: 0.793, F1: 0.773, PR AUC: 0.851
AU_2: precision: 0.744, recall: 0.744, F1: 0.744, PR AUC: 0.825
AU_4: precision: 0.744, recall: 0.582, F1: 0.653, PR AUC: 0.754
AU_5: precision: 0.818, recall: 0.730, F1: 0.771, PR AUC: 0.820
AU_6: precision: 0.696, recall: 0.800, F1: 0.744, PR AUC: 0.813
AU_7: precision: 0.600, recall: 0.417, F1: 0.492, PR AUC: 0.569
AU_9: precision: 0.917, recall: 0.917, F1: 0.917, PR AUC: 0.934
AU_10: precision: 0.000, recall: 0.000, F1: 0.000, PR AUC: 0.087
AU_12: precision: 0.822, recall: 0.902, F1: 0.860, PR AUC: 0.930
AU_14: precision: 0.500, recall: 0.556, F1: 0.526, PR AUC: 0.400
AU_15: precision: 0.500, recall: 0.370, F1: 0.426, PR AUC: 0.533
AU_17: precision: 0.781, recall: 0.794, F1: 0.787, PR AUC: 0.878
AU_20: precision: 0.654, recall: 0.739, F1: 0.694, PR AUC: 0.774
AU_23/24: precision: 0.500, recall: 0.296, F1: 0.372, PR AUC: 0.410
AU_25: precision: 0.911, recall: 0.911, F1: 0.911, PR AUC: 0.976
AU_26/27: precision: 0.850, r

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 1,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint3.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if 'fc' in name:
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.6332, Time (s): 176


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.4333, Val Acc: 83.8%, Time(s): 255.21
Best Loss: Saving Model


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f2cfc05e0f0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f2cfc05e0f0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    sel

Epoch [2/12], Loss: 0.3856, Time (s): 421


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f2cfc05e0f0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f2cfc05e0f0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    sel

Epoch [2/12], Val Loss: 0.3281, Val Acc: 88.6%, Time(s): 495.91
Best Loss: Saving Model
Epoch [3/12], Loss: 0.2801, Time (s): 658
Epoch [3/12], Val Loss: 0.2764, Val Acc: 90.1%, Time(s): 731.43
Best Loss: Saving Model
Epoch [4/12], Loss: 0.2473, Time (s): 895
Epoch [4/12], Val Loss: 0.2707, Val Acc: 90.9%, Time(s): 971.19
Best Loss: Saving Model
Epoch [5/12], Loss: 0.2353, Time (s): 1141
Epoch [5/12], Val Loss: 0.2478, Val Acc: 91.4%, Time(s): 1218.39
Best Loss: Saving Model
Epoch [6/12], Loss: 0.2269, Time (s): 1381
Epoch [6/12], Val Loss: 0.2402, Val Acc: 91.6%, Time(s): 1455.59
Best Loss: Saving Model
Epoch [7/12], Loss: 0.2219, Time (s): 1617
Epoch [7/12], Val Loss: 0.2422, Val Acc: 92.0%, Time(s): 1690.29
Epoch [8/12], Loss: 0.2144, Time (s): 1852
Epoch [8/12], Val Loss: 0.2259, Val Acc: 91.9%, Time(s): 1925.24
Best Loss: Saving Model
Epoch [9/12], Loss: 0.2084, Time (s): 2087
Epoch [9/12], Val Loss: 0.2218, Val Acc: 92.2%, Time(s): 2166.73
Best Loss: Saving Model
Epoch [10/12], L

In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 2,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint4.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if 'fc' in name:
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.6709, Time (s): 177


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.4006, Val Acc: 85.5%, Time(s): 270.76
Best Loss: Saving Model
Epoch [2/12], Loss: 0.4168, Time (s): 449
Epoch [2/12], Val Loss: 0.2792, Val Acc: 89.8%, Time(s): 542.40
Best Loss: Saving Model
Epoch [3/12], Loss: 0.3023, Time (s): 720
Epoch [3/12], Val Loss: 0.2375, Val Acc: 91.1%, Time(s): 814.81
Best Loss: Saving Model
Epoch [4/12], Loss: 0.2677, Time (s): 994
Epoch [4/12], Val Loss: 0.2235, Val Acc: 92.0%, Time(s): 1089.30
Best Loss: Saving Model
Epoch [5/12], Loss: 0.2619, Time (s): 1267
Epoch [5/12], Val Loss: 0.2167, Val Acc: 92.3%, Time(s): 1357.81
Best Loss: Saving Model
Epoch [6/12], Loss: 0.2390, Time (s): 1528
Epoch [6/12], Val Loss: 0.2109, Val Acc: 91.9%, Time(s): 1615.82
Best Loss: Saving Model
Epoch [7/12], Loss: 0.2363, Time (s): 1780
Epoch [7/12], Val Loss: 0.2041, Val Acc: 92.6%, Time(s): 1866.33
Best Loss: Saving Model
Epoch [8/12], Loss: 0.2252, Time (s): 2026
Epoch [8/12], Val Loss: 0.2030, Val Acc: 92.7%, Time(s): 2112.04
Best Loss: Saving

In [ ]:
# Validation Loss
print(f'Epoch 1: {np.mean([0.4703, 0.4333, 0.4006])} +- {np.std([0.4703, 0.4333, 0.4006])} ')
print(f'Epoch 2: {np.mean([0.3563, 0.3281, 0.2792])} +- {np.std([0.3563, 0.3281, 0.2792])} ')
print(f'Epoch 3: {np.mean([0.2828, 0.2764, 0.2375])} +- {np.std([0.2828, 0.2764, 0.2375])} ')
print(f'Epoch 4: {np.mean([0.2599, 0.2707, 0.2235])} +- {np.std([0.2599, 0.2707, 0.2235])} ')
print(f'Epoch 5: {np.mean([0.2531, 0.2478, 0.2167])} +- {np.std([0.2531, 0.2478, 0.2167])} ')
print(f'Epoch 6: {np.mean([0.2421, 0.2402, 0.2109])} +- {np.std([0.2421, 0.2402, 0.2109])} ')
print(f'Epoch 7: {np.mean([0.2360, 0.2422, 0.2041])} +- {np.std([0.2360, 0.2422, 0.2041])} ')
print(f'Epoch 8: {np.mean([0.2309, 0.2259, 0.2030])} +- {np.std([0.2309, 0.2259, 0.2030])} ')
print(f'Epoch 9: {np.mean([0.2309, 0.2218, 0.1978])} +- {np.std([0.2309, 0.2218, 0.1978])} ')
print(f'Epoch 10: {np.mean([0.2255, 0.2125, 0.1949])} +- {np.std([0.2255, 0.2125, 0.1949])} ')
print(f'Epoch 11: {np.mean([0.2212, 0.2182, 0.1933])} +- {np.std([0.2212, 0.2182, 0.1933])} ')
print(f'Epoch 12: {np.mean([0.2176, 0.2129, 0.1900])} +- {np.std([0.2176, 0.2129, 0.1900])} ')

Epoch 1: 0.43473333333333336 +- 0.028472950126196765 
Epoch 2: 0.3212 +- 0.03185184453057625 
Epoch 3: 0.2655666666666667 +- 0.02001738133611776 
Epoch 4: 0.2513666666666667 +- 0.02019196759990357 
Epoch 5: 0.2392 +- 0.016056359072550243 
Epoch 6: 0.23106666666666667 +- 0.014281067498226064 
Epoch 7: 0.2274333333333333 +- 0.016692180471373077 
Epoch 8: 0.21993333333333331 +- 0.012146421512344914 
Epoch 9: 0.21683333333333332 +- 0.013961932371830034 
Epoch 10: 0.21096666666666666 +- 0.012539360253041448 
Epoch 11: 0.2109 +- 0.012505198918849715 
Epoch 12: 0.20683333333333334 +- 0.012056625656551759 


In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint3.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=1, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.765, recall: 0.736, F1: 0.750, PR AUC: 0.770
AU_2: precision: 0.829, recall: 0.829, F1: 0.829, PR AUC: 0.866
AU_4: precision: 0.656, recall: 0.824, F1: 0.730, PR AUC: 0.808
AU_5: precision: 0.704, recall: 0.792, F1: 0.745, PR AUC: 0.699
AU_6: precision: 0.640, recall: 0.500, F1: 0.561, PR AUC: 0.710
AU_7: precision: 0.500, recall: 0.500, F1: 0.500, PR AUC: 0.478
AU_9: precision: 0.895, recall: 0.944, F1: 0.919, PR AUC: 0.948
AU_10: precision: 0.000, recall: 0.000, F1: 0.000, PR AUC: 0.114
AU_12: precision: 0.938, recall: 0.833, F1: 0.882, PR AUC: 0.971
AU_14: precision: 0.167, recall: 0.125, F1: 0.143, PR AUC: 0.159
AU_15: precision: 0.667, recall: 0.316, F1: 0.429, PR AUC: 0.496
AU_17: precision: 0.797, recall: 0.839, F1: 0.817, PR AUC: 0.910
AU_20: precision: 0.583, recall: 0.636, F1: 0.609, PR AUC: 0.556
AU_23/24: precision: 0.615, recall: 0.296, F1: 0.400, PR AUC: 0.553
AU_25: precision: 0.952, recall: 0.878, F1: 0.913, PR AUC: 0.959
AU_26/27: precision: 0.771, r

In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint4.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=2, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.722, recall: 0.765, F1: 0.743, PR AUC: 0.834
AU_2: precision: 0.784, recall: 0.935, F1: 0.853, PR AUC: 0.942
AU_4: precision: 0.844, recall: 0.915, F1: 0.878, PR AUC: 0.907
AU_5: precision: 0.654, recall: 0.586, F1: 0.618, PR AUC: 0.543
AU_6: precision: 0.700, recall: 0.761, F1: 0.729, PR AUC: 0.826
AU_7: precision: 0.582, recall: 0.744, F1: 0.653, PR AUC: 0.669
AU_9: precision: 0.893, recall: 0.893, F1: 0.893, PR AUC: 0.972
AU_10: precision: 0.000, recall: 0.000, F1: 0.000, PR AUC: 0.053
AU_12: precision: 0.950, recall: 0.864, F1: 0.905, PR AUC: 0.962
AU_14: precision: 0.333, recall: 0.077, F1: 0.125, PR AUC: 0.240
AU_15: precision: 0.333, recall: 0.588, F1: 0.426, PR AUC: 0.403
AU_17: precision: 0.792, recall: 0.905, F1: 0.844, PR AUC: 0.929
AU_20: precision: 0.667, recall: 0.538, F1: 0.596, PR AUC: 0.639
AU_23/24: precision: 0.314, recall: 0.500, F1: 0.386, PR AUC: 0.303
AU_25: precision: 0.901, recall: 0.867, F1: 0.883, PR AUC: 0.979
AU_26/27: precision: 0.805, r

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


## **Second Fold**: Unfreeze FC layers and last residual block

In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 1,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint1.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if ('fc' in name) or ('layer4' in name):
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.5048, Time (s): 174


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.2461, Val Acc: 91.2%, Time(s): 253.99
Best Loss: Saving Model
Epoch [2/12], Loss: 0.1804, Time (s): 435
Epoch [2/12], Val Loss: 0.1941, Val Acc: 92.9%, Time(s): 516.98
Best Loss: Saving Model
Epoch [3/12], Loss: 0.1325, Time (s): 696
Epoch [3/12], Val Loss: 0.1713, Val Acc: 93.4%, Time(s): 777.60
Best Loss: Saving Model
Epoch [4/12], Loss: 0.1098, Time (s): 955
Epoch [4/12], Val Loss: 0.1712, Val Acc: 93.6%, Time(s): 1035.48
Best Loss: Saving Model
Epoch [5/12], Loss: 0.0883, Time (s): 1211
Epoch [5/12], Val Loss: 0.1701, Val Acc: 93.5%, Time(s): 1290.34
Best Loss: Saving Model
Epoch [6/12], Loss: 0.0729, Time (s): 1469
Epoch [6/12], Val Loss: 0.1669, Val Acc: 93.4%, Time(s): 1550.10
Best Loss: Saving Model
Epoch [7/12], Loss: 0.0623, Time (s): 1728
Epoch [7/12], Val Loss: 0.1667, Val Acc: 93.8%, Time(s): 1809.69
Best Loss: Saving Model
Epoch [8/12], Loss: 0.0502, Time (s): 1986
Epoch [8/12], Val Loss: 0.1657, Val Acc: 93.6%, Time(s): 2066.90
Best Loss: Saving

In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 0,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint5.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if ('fc' in name) or ('layer4' in name):
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.5009, Time (s): 174


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.2303, Val Acc: 91.9%, Time(s): 261.50
Best Loss: Saving Model
Epoch [2/12], Loss: 0.1760, Time (s): 428
Epoch [2/12], Val Loss: 0.1968, Val Acc: 93.3%, Time(s): 512.83
Best Loss: Saving Model
Epoch [3/12], Loss: 0.1299, Time (s): 676
Epoch [3/12], Val Loss: 0.1796, Val Acc: 93.7%, Time(s): 760.43
Best Loss: Saving Model
Epoch [4/12], Loss: 0.1044, Time (s): 923
Epoch [4/12], Val Loss: 0.1735, Val Acc: 94.0%, Time(s): 1006.92
Best Loss: Saving Model
Epoch [5/12], Loss: 0.0858, Time (s): 1170
Epoch [5/12], Val Loss: 0.1727, Val Acc: 93.8%, Time(s): 1255.94
Best Loss: Saving Model
Epoch [6/12], Loss: 0.0768, Time (s): 1421
Epoch [6/12], Val Loss: 0.1723, Val Acc: 93.6%, Time(s): 1505.89
Best Loss: Saving Model
Epoch [7/12], Loss: 0.0610, Time (s): 1674
Epoch [7/12], Val Loss: 0.1727, Val Acc: 93.7%, Time(s): 1759.58
Epoch [8/12], Loss: 0.0488, Time (s): 1928
Epoch [8/12], Val Loss: 0.1726, Val Acc: 93.8%, Time(s): 2012.77
Epoch [9/12], Loss: 0.0412, Time (s): 217

In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 2,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint6.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if ('fc' in name) or ('layer4' in name):
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.5173, Time (s): 161


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.2095, Val Acc: 92.4%, Time(s): 247.13
Best Loss: Saving Model
Epoch [2/12], Loss: 0.1848, Time (s): 409
Epoch [2/12], Val Loss: 0.1701, Val Acc: 93.5%, Time(s): 495.31
Best Loss: Saving Model
Epoch [3/12], Loss: 0.1363, Time (s): 659
Epoch [3/12], Val Loss: 0.1585, Val Acc: 94.4%, Time(s): 746.08
Best Loss: Saving Model
Epoch [4/12], Loss: 0.1123, Time (s): 912
Epoch [4/12], Val Loss: 0.1581, Val Acc: 94.1%, Time(s): 998.92
Best Loss: Saving Model
Epoch [5/12], Loss: 0.0920, Time (s): 1165
Epoch [5/12], Val Loss: 0.1522, Val Acc: 94.3%, Time(s): 1253.69
Best Loss: Saving Model
Epoch [6/12], Loss: 0.0807, Time (s): 1421
Epoch [6/12], Val Loss: 0.1523, Val Acc: 94.3%, Time(s): 1510.05
Epoch [7/12], Loss: 0.0674, Time (s): 1679
Epoch [7/12], Val Loss: 0.1514, Val Acc: 94.2%, Time(s): 1769.44
Best Loss: Saving Model
Epoch [8/12], Loss: 0.0573, Time (s): 1941
Epoch [8/12], Val Loss: 0.1524, Val Acc: 94.2%, Time(s): 2032.03
Epoch [9/12], Loss: 0.0449, Time (s): 2204

In [ ]:
print(f'Epoch 1: {np.mean([0.2461, 0.2303, 0.2095])} +- {np.std([0.2461, 0.2303, 0.2095])} ')
print(f'Epoch 2: {np.mean([0.1941, 0.1968, 0.1701])} +- {np.std([0.1941, 0.1968, 0.1701])} ')
print(f'Epoch 3: {np.mean([0.1713, 0.1796, 0.1585])} +- {np.std([0.1713, 0.1796, 0.1585])} ')
print(f'Epoch 4: {np.mean([0.1712, 0.1735, 0.1581])} +- {np.std([0.1712, 0.1735, 0.1581])} ')
print(f'Epoch 5: {np.mean([0.1701, 0.1727, 0.1522])} +- {np.std([0.1701, 0.1727, 0.1522])} ')
print(f'Epoch 6: {np.mean([0.1669, 0.1723, 0.1523])} +- {np.std([0.1669, 0.1723, 0.1523])} ')
print(f'Epoch 7: {np.mean([0.1667, 0.1727, 0.1514])} +- {np.std([0.1667, 0.1727, 0.1514])} ')
print(f'Epoch 8: {np.mean([0.1657, 0.1726, 0.1524])} +- {np.std([0.1657, 0.1726, 0.1524])} ')
print(f'Epoch 9: {np.mean([0.1689, 0.1768, 0.1490])} +- {np.std([0.1689, 0.1768, 0.1490])} ')
print(f'Epoch 10: {np.mean([0.1614, 0.1746, 0.1525])} +- {np.std([0.1614, 0.1746, 0.1525])} ')
print(f'Epoch 11: {np.mean([0.1683, 0.1776, 0.1553])} +- {np.std([0.1683, 0.1776, 0.1553])} ')
print(f'Epoch 12: {np.mean([0.1779, 0.1792, 0.1575])} +- {np.std([0.1779, 0.1792, 0.1575])} ')

Epoch 1: 0.22863333333333336 +- 0.01498829172684096 
Epoch 2: 0.18700000000000003 +- 0.012000833304400157 
Epoch 3: 0.16979999999999998 +- 0.0086790936546777 
Epoch 4: 0.1676 +- 0.006782821438506742 
Epoch 5: 0.165 +- 0.00911299438530863 
Epoch 6: 0.16383333333333333 +- 0.008448010942753862 
Epoch 7: 0.1636 +- 0.008967719888578137 
Epoch 8: 0.16356666666666667 +- 0.008383449303372818 
Epoch 9: 0.16490000000000002 +- 0.01169643820428539 
Epoch 10: 0.1628333333333333 +- 0.009079035680560406 
Epoch 11: 0.16706666666666667 +- 0.009145612184114431 
Epoch 12: 0.17153333333333332 +- 0.009937247550951364 


In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint5.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=0, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.895, recall: 0.879, F1: 0.887, PR AUC: 0.917
AU_2: precision: 0.750, recall: 0.769, F1: 0.759, PR AUC: 0.888
AU_4: precision: 0.725, recall: 0.673, F1: 0.698, PR AUC: 0.796
AU_5: precision: 0.909, recall: 0.811, F1: 0.857, PR AUC: 0.932
AU_6: precision: 0.732, recall: 0.750, F1: 0.741, PR AUC: 0.831
AU_7: precision: 0.655, recall: 0.528, F1: 0.585, PR AUC: 0.575
AU_9: precision: 0.957, recall: 0.917, F1: 0.936, PR AUC: 0.962
AU_10: precision: 0.000, recall: 0.000, F1: 0.000, PR AUC: 0.073
AU_12: precision: 0.792, recall: 0.927, F1: 0.854, PR AUC: 0.942
AU_14: precision: 0.545, recall: 0.667, F1: 0.600, PR AUC: 0.733
AU_15: precision: 0.576, recall: 0.704, F1: 0.633, PR AUC: 0.752
AU_17: precision: 0.836, recall: 0.889, F1: 0.862, PR AUC: 0.934
AU_20: precision: 0.857, recall: 0.783, F1: 0.818, PR AUC: 0.914
AU_23/24: precision: 0.625, recall: 0.370, F1: 0.465, PR AUC: 0.607
AU_25: precision: 0.960, recall: 0.941, F1: 0.950, PR AUC: 0.984
AU_26/27: precision: 0.864, r

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint1.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=1, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.745, recall: 0.774, F1: 0.759, PR AUC: 0.889
AU_2: precision: 0.744, recall: 0.829, F1: 0.784, PR AUC: 0.893
AU_4: precision: 0.703, recall: 0.882, F1: 0.783, PR AUC: 0.825
AU_5: precision: 0.733, recall: 0.917, F1: 0.815, PR AUC: 0.854
AU_6: precision: 0.594, recall: 0.594, F1: 0.594, PR AUC: 0.769
AU_7: precision: 0.618, recall: 0.618, F1: 0.618, PR AUC: 0.517
AU_9: precision: 0.941, recall: 0.889, F1: 0.914, PR AUC: 0.950
AU_10: precision: 0.000, recall: 0.000, F1: 0.000, PR AUC: 0.216
AU_12: precision: 0.886, recall: 0.861, F1: 0.873, PR AUC: 0.957
AU_14: precision: 0.429, recall: 0.375, F1: 0.400, PR AUC: 0.386
AU_15: precision: 0.893, recall: 0.658, F1: 0.758, PR AUC: 0.898
AU_17: precision: 0.926, recall: 0.893, F1: 0.909, PR AUC: 0.962
AU_20: precision: 0.789, recall: 0.682, F1: 0.732, PR AUC: 0.751
AU_23/24: precision: 0.762, recall: 0.593, F1: 0.667, PR AUC: 0.786
AU_25: precision: 0.925, recall: 0.956, F1: 0.940, PR AUC: 0.962
AU_26/27: precision: 0.811, r

In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint6.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=2, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.727, recall: 0.941, F1: 0.821, PR AUC: 0.913
AU_2: precision: 0.909, recall: 0.968, F1: 0.937, PR AUC: 0.984
AU_4: precision: 0.849, recall: 0.873, F1: 0.861, PR AUC: 0.919
AU_5: precision: 0.750, recall: 0.621, F1: 0.679, PR AUC: 0.712
AU_6: precision: 0.712, recall: 0.804, F1: 0.755, PR AUC: 0.874
AU_7: precision: 0.660, recall: 0.814, F1: 0.729, PR AUC: 0.705
AU_9: precision: 0.963, recall: 0.929, F1: 0.945, PR AUC: 0.988
AU_10: precision: 0.667, recall: 0.400, F1: 0.500, PR AUC: 0.208
AU_12: precision: 0.950, recall: 0.864, F1: 0.905, PR AUC: 0.953
AU_14: precision: 0.571, recall: 0.308, F1: 0.400, PR AUC: 0.478
AU_15: precision: 0.444, recall: 0.706, F1: 0.545, PR AUC: 0.762
AU_17: precision: 0.859, recall: 0.873, F1: 0.866, PR AUC: 0.945
AU_20: precision: 0.792, recall: 0.731, F1: 0.760, PR AUC: 0.847
AU_23/24: precision: 0.667, recall: 0.636, F1: 0.651, PR AUC: 0.738
AU_25: precision: 0.960, recall: 0.914, F1: 0.937, PR AUC: 0.988
AU_26/27: precision: 0.837, r

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


## **Third Fold**: Unfreeze FC layers and last two residual blocks

In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 2,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint2.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if ('fc' in name) or ('layer4' in name) or ('layer3' in name):
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.5075, Time (s): 174


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.2039, Val Acc: 92.1%, Time(s): 265.46
Best Loss: Saving Model
Epoch [2/12], Loss: 0.1639, Time (s): 441
Epoch [2/12], Val Loss: 0.1615, Val Acc: 94.0%, Time(s): 530.23
Best Loss: Saving Model
Epoch [3/12], Loss: 0.1203, Time (s): 695
Epoch [3/12], Val Loss: 0.1573, Val Acc: 93.8%, Time(s): 781.09
Best Loss: Saving Model
Epoch [4/12], Loss: 0.0999, Time (s): 944
Epoch [4/12], Val Loss: 0.1510, Val Acc: 94.2%, Time(s): 1028.95
Best Loss: Saving Model
Epoch [5/12], Loss: 0.0784, Time (s): 1189
Epoch [5/12], Val Loss: 0.1531, Val Acc: 94.4%, Time(s): 1272.84
Epoch [6/12], Loss: 0.0641, Time (s): 1431
Epoch [6/12], Val Loss: 0.1502, Val Acc: 94.3%, Time(s): 1515.18
Best Loss: Saving Model
Epoch [7/12], Loss: 0.0617, Time (s): 1674
Epoch [7/12], Val Loss: 0.1516, Val Acc: 93.9%, Time(s): 1758.11
Epoch [8/12], Loss: 0.0409, Time (s): 1917
Epoch [8/12], Val Loss: 0.1513, Val Acc: 94.1%, Time(s): 2001.13
Epoch [9/12], Loss: 0.0398, Time (s): 2159
Epoch [9/12], Val Loss

In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 0,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint7.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if ('fc' in name) or ('layer4' in name) or ('layer3' in name):
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.4836, Time (s): 173


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.2130, Val Acc: 92.9%, Time(s): 260.89
Best Loss: Saving Model
Epoch [2/12], Loss: 0.1585, Time (s): 433
Epoch [2/12], Val Loss: 0.1882, Val Acc: 93.4%, Time(s): 521.20
Best Loss: Saving Model
Epoch [3/12], Loss: 0.1133, Time (s): 694
Epoch [3/12], Val Loss: 0.1748, Val Acc: 93.9%, Time(s): 781.94
Best Loss: Saving Model
Epoch [4/12], Loss: 0.0853, Time (s): 956
Epoch [4/12], Val Loss: 0.1675, Val Acc: 94.2%, Time(s): 1048.15
Best Loss: Saving Model
Epoch [5/12], Loss: 0.0676, Time (s): 1225
Epoch [5/12], Val Loss: 0.1674, Val Acc: 94.1%, Time(s): 1316.10
Best Loss: Saving Model
Epoch [6/12], Loss: 0.0518, Time (s): 1493
Epoch [6/12], Val Loss: 0.1669, Val Acc: 94.0%, Time(s): 1583.06
Best Loss: Saving Model
Epoch [7/12], Loss: 0.0399, Time (s): 1759
Epoch [7/12], Val Loss: 0.1668, Val Acc: 94.2%, Time(s): 1849.96
Best Loss: Saving Model
Epoch [8/12], Loss: 0.0339, Time (s): 2026
Epoch [8/12], Val Loss: 0.1687, Val Acc: 94.2%, Time(s): 2116.84
Epoch [9/12], Los

In [ ]:
args = AttrDict()
args_dict = {
    'gpu': True,
    'transform': transform,
    'fold': 1,
    'checkpoint': '/content/drive/MyDrive/training/checkpoints/checkpoint8.pt',
    'learn_rate': 0.0001,
    'batch_size': 32,
    'epochs': 12,
    'num_workers': 4,
    'resume': False 
}

args.update(args_dict)
convnet = get_model('/content/drive/My Drive/training/876.pt')

# Freeze feature map layers
for name, param in convnet.named_parameters():
  if ('fc' in name) or ('layer4' in name) or ('layer3' in name):
    param.requires_grad = True
  else:
    param.requires_grad = False
train(convnet, args=args)

Epoch [1/12], Loss: 0.4703, Time (s): 180


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/12], Val Loss: 0.2179, Val Acc: 92.2%, Time(s): 262.62
Best Loss: Saving Model
Epoch [2/12], Loss: 0.1638, Time (s): 443
Epoch [2/12], Val Loss: 0.1791, Val Acc: 93.1%, Time(s): 525.43
Best Loss: Saving Model
Epoch [3/12], Loss: 0.1152, Time (s): 707
Epoch [3/12], Val Loss: 0.1748, Val Acc: 93.7%, Time(s): 790.81
Best Loss: Saving Model
Epoch [4/12], Loss: 0.0871, Time (s): 974
Epoch [4/12], Val Loss: 0.1670, Val Acc: 93.7%, Time(s): 1056.85
Best Loss: Saving Model
Epoch [5/12], Loss: 0.0703, Time (s): 1241
Epoch [5/12], Val Loss: 0.1633, Val Acc: 93.8%, Time(s): 1324.34
Best Loss: Saving Model
Epoch [6/12], Loss: 0.0530, Time (s): 1507
Epoch [6/12], Val Loss: 0.1643, Val Acc: 94.1%, Time(s): 1590.69
Epoch [7/12], Loss: 0.0420, Time (s): 1772
Epoch [7/12], Val Loss: 0.1612, Val Acc: 93.9%, Time(s): 1855.06
Best Loss: Saving Model
Epoch [8/12], Loss: 0.0332, Time (s): 2037
Epoch [8/12], Val Loss: 0.1584, Val Acc: 93.9%, Time(s): 2119.99
Best Loss: Saving Model
Epoch [9/12], Los

In [ ]:
print(f'Epoch 1: {np.mean([0.2039, 0.2130, 0.2179])} +- {np.std([0.2039, 0.2130, 0.2179])} ')
print(f'Epoch 2: {np.mean([0.1615, 0.1882, 0.1791])} +- {np.std([0.1615, 0.1882, 0.1791])} ')
print(f'Epoch 3: {np.mean([0.1573, 0.1748, 0.1748])} +- {np.std([0.1573, 0.1748, 0.1748])} ')
print(f'Epoch 4: {np.mean([0.1510, 0.1675, 0.1670])} +- {np.std([0.1510, 0.1675, 0.1670])} ')
print(f'Epoch 5: {np.mean([0.1531, 0.1674, 0.1633])} +- {np.std([0.1531, 0.1674, 0.1633])} ')
print(f'Epoch 6: {np.mean([0.1502, 0.1669, 0.1643])} +- {np.std([0.1502, 0.1669, 0.1643])} ')
print(f'Epoch 7: {np.mean([0.1516, 0.1668, 0.1612])} +- {np.std([0.1516, 0.1668, 0.1612])} ')
print(f'Epoch 8: {np.mean([0.1513, 0.1687, 0.1584])} +- {np.std([0.1513, 0.1687, 0.1584])} ')
print(f'Epoch 9: {np.mean([0.1468, 0.1698, 0.1624])} +- {np.std([0.1468, 0.1698, 0.1624])} ')
print(f'Epoch 10: {np.mean([0.1541, 0.1717, 0.1673])} +- {np.std([0.1541, 0.1717, 0.1673])} ')
print(f'Epoch 11: {np.mean([0.1496, 0.1707, 0.1691])} +- {np.std([0.1496, 0.1707, 0.0174])} ')
print(f'Epoch 12: {np.mean([0.1510, 0.1741, 0.1724])} +- {np.std([0.1510, 0.1741, 0.1724])} ')

Epoch 1: 0.2116 +- 0.00580057468417283 
Epoch 2: 0.17626666666666668 +- 0.011082819536963006 
Epoch 3: 0.16896666666666668 +- 0.008249579113843062 
Epoch 4: 0.16183333333333336 +- 0.007663042621715106 
Epoch 5: 0.16126666666666667 +- 0.006012394605220851 
Epoch 6: 0.16046666666666667 +- 0.007336817354199903 
Epoch 7: 0.15986666666666668 +- 0.006276587891167901 
Epoch 8: 0.15946666666666665 +- 0.0071434507689833535 
Epoch 9: 0.15966666666666665 +- 0.009586564672614593 
Epoch 10: 0.16436666666666666 +- 0.007478561953269422 
Epoch 11: 0.11256666666666666 +- 0.06784208788715813 
Epoch 12: 0.16583333333333333 +- 0.010511686618024513 


In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint7.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=0, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.912, recall: 0.897, F1: 0.904, PR AUC: 0.929
AU_2: precision: 0.756, recall: 0.795, F1: 0.775, PR AUC: 0.887
AU_4: precision: 0.812, recall: 0.709, F1: 0.757, PR AUC: 0.833
AU_5: precision: 1.000, recall: 0.757, F1: 0.862, PR AUC: 0.954
AU_6: precision: 0.718, recall: 0.700, F1: 0.709, PR AUC: 0.818
AU_7: precision: 0.654, recall: 0.472, F1: 0.548, PR AUC: 0.583
AU_9: precision: 0.957, recall: 0.917, F1: 0.936, PR AUC: 0.959
AU_10: precision: 0.000, recall: 0.000, F1: 0.000, PR AUC: 0.072
AU_12: precision: 0.796, recall: 0.951, F1: 0.867, PR AUC: 0.936
AU_14: precision: 0.714, recall: 0.556, F1: 0.625, PR AUC: 0.694
AU_15: precision: 0.600, recall: 0.778, F1: 0.677, PR AUC: 0.782
AU_17: precision: 0.864, recall: 0.905, F1: 0.884, PR AUC: 0.935
AU_20: precision: 0.760, recall: 0.826, F1: 0.792, PR AUC: 0.918
AU_23/24: precision: 0.684, recall: 0.481, F1: 0.565, PR AUC: 0.672
AU_25: precision: 0.950, recall: 0.950, F1: 0.950, PR AUC: 0.982
AU_26/27: precision: 0.884, r

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [ ]:
model = get_model('/content/drive/My Drive/training/876.pt')
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint8.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=1, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.784, recall: 0.755, F1: 0.769, PR AUC: 0.889
AU_2: precision: 0.763, recall: 0.829, F1: 0.795, PR AUC: 0.889
AU_4: precision: 0.667, recall: 0.863, F1: 0.752, PR AUC: 0.847
AU_5: precision: 0.786, recall: 0.917, F1: 0.846, PR AUC: 0.922
AU_6: precision: 0.690, recall: 0.625, F1: 0.656, PR AUC: 0.794
AU_7: precision: 0.667, recall: 0.588, F1: 0.625, PR AUC: 0.544
AU_9: precision: 1.000, recall: 0.944, F1: 0.971, PR AUC: 0.953
AU_10: precision: 0.000, recall: 0.000, F1: 0.000, PR AUC: 0.179
AU_12: precision: 0.909, recall: 0.833, F1: 0.870, PR AUC: 0.956
AU_14: precision: 0.333, recall: 0.250, F1: 0.286, PR AUC: 0.425
AU_15: precision: 0.963, recall: 0.684, F1: 0.800, PR AUC: 0.913
AU_17: precision: 0.922, recall: 0.839, F1: 0.879, PR AUC: 0.965
AU_20: precision: 0.696, recall: 0.727, F1: 0.711, PR AUC: 0.775
AU_23/24: precision: 0.737, recall: 0.519, F1: 0.609, PR AUC: 0.764
AU_25: precision: 0.914, recall: 0.944, F1: 0.929, PR AUC: 0.957
AU_26/27: precision: 0.811, r

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/training/checkpoints/checkpoint2.pt')
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

# Instantiate dataloader
dataset = AUDataset(mode='validate', fold=2, transform=transform)
data = DataLoader(dataset, 32, collate_fn=collate_fn, shuffle=True, 
           num_workers=4)

# Compute scores
precision, recall, f1, roc_auc = get_metrics(model, data)
for i in range(18):
  print(f'{dataset.label_cols[i]}: precision: {precision[i]:.3f}, recall: {recall[i]:.3f}, F1: {f1[i]:.3f}, PR AUC: {roc_auc[i]:.3f}')
print(f'mean precision: {np.mean(precision):.3f}, mean recall: {np.mean(recall):.3f}, mean F1: {np.mean(f1):.3f}, mean PR-AUC: {np.mean(roc_auc):.3f}')

AU_1: precision: 0.746, recall: 0.863, F1: 0.800, PR AUC: 0.897
AU_2: precision: 0.909, recall: 0.968, F1: 0.937, PR AUC: 0.983
AU_4: precision: 0.853, recall: 0.817, F1: 0.835, PR AUC: 0.909
AU_5: precision: 0.700, recall: 0.724, F1: 0.712, PR AUC: 0.774
AU_6: precision: 0.717, recall: 0.717, F1: 0.717, PR AUC: 0.858
AU_7: precision: 0.630, recall: 0.674, F1: 0.652, PR AUC: 0.696
AU_9: precision: 0.962, recall: 0.893, F1: 0.926, PR AUC: 0.991
AU_10: precision: 0.500, recall: 0.400, F1: 0.444, PR AUC: 0.175
AU_12: precision: 0.951, recall: 0.886, F1: 0.918, PR AUC: 0.962
AU_14: precision: 0.600, recall: 0.231, F1: 0.333, PR AUC: 0.469
AU_15: precision: 0.522, recall: 0.706, F1: 0.600, PR AUC: 0.776
AU_17: precision: 0.915, recall: 0.857, F1: 0.885, PR AUC: 0.945
AU_20: precision: 0.952, recall: 0.769, F1: 0.851, PR AUC: 0.871
AU_23/24: precision: 0.765, recall: 0.591, F1: 0.667, PR AUC: 0.771
AU_25: precision: 0.960, recall: 0.905, F1: 0.931, PR AUC: 0.990
AU_26/27: precision: 0.881, r

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
